In [19]:
%matplotlib inline

import logging, io, json, warnings
logging.basicConfig(level="INFO")
warnings.filterwarnings('ignore')

def pprint(o):
    # small helper to make dict dumps a bit prettier
    print(json.dumps(o, indent=2))

In [20]:
import sys
python = sys.executable

# as well as install a language model:
!{python} -m spacy download en_core_web_md
!{python} -m spacy link en_core_web_md en --force;

symbolic link created for C:\Users\Home\Anaconda2\envs\tensorflow\lib\site-packages\spacy\data\en_core_web_md <<===>> C:\Users\Home\Anaconda2\envs\tensorflow\lib\site-packages\en_core_web_md

    Linking successful
    C:\Users\Home\Anaconda2\envs\tensorflow\lib\site-packages\en_core_web_md
    -->
    C:\Users\Home\Anaconda2\envs\tensorflow\lib\site-packages\spacy\data\en_core_web_md

    You can now load the model via spacy.load('en_core_web_md')



usage: spacy link [-h] [-f] origin link_name [model_path]
spacy link: error: unrecognized arguments: --force;


In [21]:
import rasa_nlu
import rasa_core
import spacy

print("rasa_nlu: {} rasa_core: {}".format(rasa_nlu.__version__, rasa_core.__version__))
print("Loading spaCy language model...")
print(spacy.load("en")("Hello world!"))

rasa_nlu: 0.11.5 rasa_core: 0.10.3
Loading spaCy language model...
Hello world!


In [22]:
nlu_md = """
## intent:greet
- hey
- hello there
- hi
- hello there
- good morning
- good evening
- moin
- hey there
- let's go
- hey dude
- goodmorning
- goodevening
- good afternoon

## intent:goodbye
- cu
- good by
- cee you later
- good night
- good afternoon
- bye
- goodbye
- have a nice day
- see you around
- bye bye
- see you later

## intent:mood_affirm
- yes
- indeed
- of course
- that sounds good
- correct

## intent:mood_deny
- no
- never
- I don't think so
- don't like that
- no way
- not really

## intent:mood_great
- perfect
- very good
- great
- amazing
- feeling like a king
- wonderful
- I am feeling very good
- I am great
- I am amazing
- I am going to save the world
- super
- extremely good
- so so perfect
- so good
- so perfect

## intent:mood_unhappy
- my day was horrible
- I am sad
- I don't feel very well
- I am disappointed
- super sad
- I'm so sad
- sad
- very sad
- unhappy
- bad
- very bad
- awful
- terrible
- not so good
- not very good
- extremly sad
- so saad
- Quite bad - can I get a cute picture of a [bird](group:birds), please?
- Really bad and only [doggo](group:shibes) pics and change that.
- Not good. The only thing that could make me fell better is a picture of a cute [kitten](group:cats).
- so sad. Only the picture of a [puppy](group:shibes) could make it better.
- I am very sad. I need a [cat](group:cats) picture.
- Extremely sad. Only the cute [doggo](group:shibes) pics can make me feel better.
- Bad. Please show me a [bird](group:birds) pic!
- Pretty bad to be honest. Can you show me a [puppy](group:shibes) picture to make me fell better?

## intent: inform
- A [dog](group:shibes)
- [dog](group:shibes)
- [bird](group:birds)
- a [cat](group:cats)
- [cat](group:cats)
- a [bird](group:birds)
- of a [dog](group:shibes)
- of a [cat](group:cats)
- a [bird](group:birds), please
- a [dog](group:shibes), please
"""

%store nlu_md > nlu.md

Writing 'nlu_md' (str) to file 'nlu.md'.


In [2]:
config = """
{
"language": "en", 
"pipeline": "spacy_sklearn"
}
"""

%store config > config.json

Writing 'config' (str) to file 'config.json'.


In [23]:
import yaml


In [24]:
config = """
{
"language": "en", 
"pipeline": "spacy_sklearn"
}
"""

%store config > config.json

Writing 'config' (str) to file 'config.json'.


In [25]:
from rasa_nlu.converters import load_data
from rasa_nlu.config import RasaNLUConfig
from rasa_nlu.model import Trainer
from rasa_nlu.model import Metadata, Interpreter

# loading the nlu training samples
training_data = load_data('nlu.md')

# trainer to educate our pipeline
trainer = Trainer(RasaNLUConfig('config.json'))

# train the model!
interpreter = trainer.train(training_data)

# store it for future use
model_directory = trainer.persist("./models/nlu", fixed_model_name="current")

INFO:rasa_nlu.converters:Training data format at nlu.md is md
INFO:rasa_nlu.training_data:Training data stats: 
	- intent examples: 85 (7 distinct intents)
	- found intents: 'goodbye', 'greet', 'inform', 'mood_affirm', 'mood_deny', 'mood_great', 'mood_unhappy'
	- entity examples: 18 (1 distinct entities)
	- found entities: 'group'

INFO:rasa_nlu.utils.spacy_utils:Trying to load spacy model with name 'en'
INFO:rasa_nlu.components:Added 'nlp_spacy' to component cache. Key 'nlp_spacy-en'.
INFO:rasa_nlu.model:Starting to train component nlp_spacy
INFO:rasa_nlu.model:Finished training component.
INFO:rasa_nlu.model:Starting to train component tokenizer_spacy
INFO:rasa_nlu.model:Finished training component.
INFO:rasa_nlu.model:Starting to train component intent_featurizer_spacy
INFO:rasa_nlu.model:Finished training component.
INFO:rasa_nlu.model:Starting to train component intent_entity_featurizer_regex
INFO:rasa_nlu.model:Finished training component.
INFO:rasa_nlu.model:Starting to train co

Fitting 2 folds for each of 6 candidates, totalling 12 fits


[Parallel(n_jobs=1)]: Done  12 out of  12 | elapsed:    0.2s finished
INFO:rasa_nlu.model:Finished training component.
INFO:rasa_nlu.model:Successfully saved model into 'C:\Users\Home\nlp_combined_project\models\nlu\default\current'


In [26]:
print(interpreter.parse("I am sad, plased send me a picture of a dog"))

{'intent': {'name': 'mood_unhappy', 'confidence': 0.4413529340033879}, 'entities': [{'start': 40, 'end': 43, 'value': 'shibes', 'entity': 'group', 'extractor': 'ner_crf', 'processors': ['ner_synonyms']}], 'intent_ranking': [{'name': 'mood_unhappy', 'confidence': 0.4413529340033879}, {'name': 'mood_great', 'confidence': 0.14306296298493912}, {'name': 'goodbye', 'confidence': 0.1307660850133933}, {'name': 'inform', 'confidence': 0.12679530013616452}, {'name': 'greet', 'confidence': 0.08124976802938856}, {'name': 'mood_affirm', 'confidence': 0.05330003972486289}, {'name': 'mood_deny', 'confidence': 0.02347291010786355}], 'text': 'I am sad, plased send me a picture of a dog'}


In [8]:
rasa_nlu.__version__

'0.11.5'

In [9]:
rasa_core.__version__

'0.10.3'

In [ ]:
print("Your bot is ready to talk! Type your messages here or send 'stop'")
print("-----------------------------------------------------------------\n\n")
while True:
    
    print("\n\n user: ") 
    a = input()
    if a == 'stop':
        break
        
    print("\n\n chatbot agent: ") 
    responses = input()
    

In [3]:
import os

os.environ["KERAS_BACKEND"] = "tensorflow"

In [12]:
# Classification models in DeepPavlov

from deeppavlov import build_model, configs

CONFIG_PATH = configs.classifiers.intents_snips

model = build_model(CONFIG_PATH, download=True)

print(model(["What is the weather in Boston today?"]))

2019-01-27 11:55:55.404 DEBUG in 'urllib3.connectionpool'['connectionpool'] at line 208: Starting new HTTP connection (1): files.deeppavlov.ai
2019-01-27 11:55:55.796 DEBUG in 'urllib3.connectionpool'['connectionpool'] at line 396: http://files.deeppavlov.ai:80 "GET /deeppavlov_data/classifiers/intents_snips_v9.tar.gz.md5 HTTP/1.1" 200 190
2019-01-27 11:55:55.892 INFO in 'deeppavlov.download'['download'] at line 115: Skipped http://files.deeppavlov.ai/deeppavlov_data/classifiers/intents_snips_v9.tar.gz download because of matching hashes
2019-01-27 11:55:55.897 DEBUG in 'urllib3.connectionpool'['connectionpool'] at line 208: Starting new HTTP connection (1): files.deeppavlov.ai
2019-01-27 11:55:56.292 DEBUG in 'urllib3.connectionpool'['connectionpool'] at line 396: http://files.deeppavlov.ai:80 "GET /datasets/snips_intents/train.csv.md5 HTTP/1.1" 200 44
2019-01-27 11:55:56.368 INFO in 'deeppavlov.download'['download'] at line 115: Skipped http://files.deeppavlov.ai/datasets/snips_inten

[['GetWeather']]


In [18]:
# Neural Morphological Tagging by deeppavlov

from deeppavlov import build_model, configs
model = build_model(configs.morpho_tagger.UD2_0.morpho_en, download=True)
sentences = ["I was walking home on an unfamiliar street.", "The girl sang in the church choir about all tired in a strange land."]
for parse in model(sentences):
    print(parse)

2019-01-27 12:18:59.439 DEBUG in 'urllib3.connectionpool'['connectionpool'] at line 208: Starting new HTTP connection (1): files.deeppavlov.ai
2019-01-27 12:18:59.836 DEBUG in 'urllib3.connectionpool'['connectionpool'] at line 396: http://files.deeppavlov.ai:80 "GET /datasets/UD2.0_source/en.tar.gz.md5 HTTP/1.1" 200 156
2019-01-27 12:18:59.851 DEBUG in 'urllib3.connectionpool'['connectionpool'] at line 208: Starting new HTTP connection (1): files.deeppavlov.ai
2019-01-27 12:19:00.285 DEBUG in 'urllib3.connectionpool'['connectionpool'] at line 396: http://files.deeppavlov.ai:80 "GET /datasets/UD2.0_source/en.tar.gz HTTP/1.1" 200 2617225
2019-01-27 12:19:00.295 INFO in 'deeppavlov.core.data.utils'['utils'] at line 64: Downloading from http://files.deeppavlov.ai/datasets/UD2.0_source/en.tar.gz to C:\Users\Home\.deeppavlov\downloads\UD2.0_source\en.tar.gz
100%|█████████████████████████████████████| 2.62M/2.62M [00:02<00:00, 1.06MB/s]
2019-01-27 12:19:02.791 INFO in 'deeppavlov.core.data.ut

2019-01-27 12:19:13.746 INFO in 'deeppavlov.models.morpho_tagger.network'['layer_utils'] at line 104:                                                                  conv2d_4[0][0]                   
2019-01-27 12:19:13.751 INFO in 'deeppavlov.models.morpho_tagger.network'['layer_utils'] at line 104:                                                                  conv2d_5[0][0]                   
2019-01-27 12:19:13.756 INFO in 'deeppavlov.models.morpho_tagger.network'['layer_utils'] at line 104:                                                                  conv2d_6[0][0]                   
2019-01-27 12:19:13.756 INFO in 'deeppavlov.models.morpho_tagger.network'['layer_utils'] at line 104:                                                                  conv2d_7[0][0]                   
2019-01-27 12:19:13.761 INFO in 'deeppavlov.models.morpho_tagger.network'['layer_utils'] at line 170: ______________________________________________________________________________________________

1	I	PRON	Case=Nom|Number=Sing|Person=1|PronType=Prs
2	was	AUX	Mood=Ind|Number=Sing|Person=1|Tense=Past|VerbForm=Fin
3	walking	VERB	Tense=Pres|VerbForm=Part
4	home	ADV	_
5	on	ADP	_
6	an	DET	Definite=Ind|PronType=Art
7	unfamiliar	ADJ	Degree=Pos
8	street	NOUN	Number=Sing
9	.	PUNCT	_

1	The	DET	Definite=Def|PronType=Art
2	girl	NOUN	Number=Sing
3	sang	NOUN	Number=Sing
4	in	ADP	_
5	the	DET	Definite=Def|PronType=Art
6	church	NOUN	Number=Sing
7	choir	NOUN	Number=Sing
8	about	ADP	_
9	all	ADV	_
10	tired	ADJ	Degree=Pos
11	in	ADP	_
12	a	DET	Definite=Ind|PronType=Art
13	strange	ADJ	Degree=Pos
14	land	NOUN	Number=Sing
15	.	PUNCT	_



In [25]:
tensorflow.reset_default_graph()

In [29]:
#Named Entity Recognition (NER) by deeppavlov

from deeppavlov import configs, build_model

ner_model = build_model(configs.ner.ner_ontonotes)
ner_model(['Bob Ross lived in Florida'])

2019-01-27 13:25:34.101 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 103: [loading vocabulary from C:\Users\Home\.deeppavlov\models\ner_ontonotes\tag.dict]
2019-01-27 13:25:34.832 INFO in 'deeppavlov.core.data.simple_vocab'['simple_vocab'] at line 103: [loading vocabulary from C:\Users\Home\.deeppavlov\models\ner_ontonotes\char.dict]
2019-01-27 13:25:35.325 INFO in 'deeppavlov.models.embedders.glove_embedder'['glove_embedder'] at line 52: [loading GloVe embeddings from `C:\Users\Home\.deeppavlov\downloads\embeddings\glove.6B.100d.txt`]
2019-01-27 13:25:35.511 INFO in 'gensim.models.utils_any2vec'['utils_any2vec'] at line 170: loading projection weights from C:\Users\Home\.deeppavlov\downloads\embeddings\glove.6B.100d.txt
2019-01-27 13:25:35.792 DEBUG in 'smart_open.smart_open_lib'['smart_open_lib'] at line 176: {'kw': {}, 'mode': 'rb', 'uri': 'C:\\Users\\Home\\.deeppavlov\\downloads\\embeddings\\glove.6B.100d.txt'}
2019-01-27 13:26:38.349 INFO in 'gensim.models.u

NotFoundError: Key LSTM_0/cudnn_bi_gru/Forward/cudnn_lstm/cudnn_lstm/rnn/multi_rnn_cell/cell_0/lstm_cell/kernel not found in checkpoint
	 [[Node: save/RestoreV2_11 = RestoreV2[dtypes=[DT_FLOAT], _device="/job:localhost/replica:0/task:0/device:CPU:0"](_arg_save/Const_0_0, save/RestoreV2_11/tensor_names, save/RestoreV2_11/shape_and_slices)]]

Caused by op 'save/RestoreV2_11', defined at:
  File "C:\Users\Home\Anaconda2\envs\tensorflow\lib\runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "C:\Users\Home\Anaconda2\envs\tensorflow\lib\runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "C:\Users\Home\Anaconda2\envs\tensorflow\lib\site-packages\ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "C:\Users\Home\Anaconda2\envs\tensorflow\lib\site-packages\traitlets\config\application.py", line 658, in launch_instance
    app.start()
  File "C:\Users\Home\Anaconda2\envs\tensorflow\lib\site-packages\ipykernel\kernelapp.py",

In [31]:
import tensorflow